<a href="https://colab.research.google.com/github/h4r1c0t/WildCodeSchool/blob/master/Odyssey/Tuto/Importer_donn%C3%A9es_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ce notebook fournit des recettes pour charger et enregistrer des données provenant de sources externes.

# Système de fichiers local

## Importer des fichiers à partir de votre système de fichiers local

<code>files.upload</code> affiche un dictionnaire des fichiers importés.
Ce dictionnaire est identifié par le nom du fichier, et les valeurs sont les données qui ont été importées.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Télécharger des fichiers sur votre système de fichiers local

<code>files.download</code> appelle le téléchargement du fichier sur votre ordinateur local via un navigateur.


In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

Vous pouvez accéder aux fichiers dans Drive de plusieurs manières. Par exemple :
- Installer votre Google Drive dans la machine virtuelle de l'environnement d'exécution
- Utiliser un wrapper autour de l'API, tel que <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">PyDrive</a>
- Utiliser l'<a href="https://developers.google.com/drive/v3/web/about-sdk">API REST native</a>



Vous trouverez ci-dessous des exemples pour chaque méthode.

## Installer Google Drive en local

L'exemple ci-dessous montre comment installer votre Google Drive dans votre environnement d'exécution à l'aide d'un code d'autorisation, et comment y écrire et lire des fichiers. Une fois le code exécuté, le nouveau fichier &#40;<code>foo.txt</code>&#41; est affiché sur le site <a href="https://drive.google.com/">https://drive.google.com/</a>.

Notez que cela n'inclut que la lecture, l'écriture et le déplacement de fichiers. Pour modifier par programmation les paramètres de partage ou d’autres métadonnées, utilisez l'une des options ci-dessous.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive

Les exemples ci-dessous illustrent l'authentification ainsi que l'importation/le téléchargement de fichiers à l'aide de PyDrive. D'autres exemples sont disponibles dans la <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">documentation concernant PyDrive</a>.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authentifiez-vous et créez le client de PyDrive.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Créez et importez un fichier texte.


In [0]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Chargez un fichier par identifiant et imprimez son contenu.


In [0]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## API REST Drive

Pour que vous puissiez utiliser l'API Drive, nous devons d'abord vous authentifier et créer un client de l'API.


In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Avec ce client, nous pouvons utiliser n’importe quelle fonction du <a href="https://developers.google.com/drive/v3/reference/">guide de référence de l'API Google Drive</a>. &#40;Voir les exemples ci-après.&#41;


### Créer un fichier Drive avec des données provenant de Python

Commencez par créer un fichier local à importer.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Importez-le en utilisant la méthode <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a>. Pour en savoir plus sur l'importation de fichiers, consultez la <a href="https://developers.google.com/drive/v3/web/manage-uploads">documentation pour les développeurs</a>.

In [0]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


Après l'exécution de la cellule ci-dessus, un nouveau fichier intitulé "Sample file" est affiché sur le site <a href="https://drive.google.com/">https://drive.google.com/</a>.

### Télécharger des données d'un fichier Drive dans Python

Téléchargez le fichier que nous avons importé ci-dessus.

In [0]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


Pour télécharger un fichier différent, attribuez une valeur à la variable <code>file&#95;id</code> ci-dessus correspondant à l'identifiant de ce fichier &#40;exemple : "1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz"&#41;.

# Google Sheets

Les exemples ci-dessous utilisent la bibliothèque Open Source <a href="https://github.com/burnash/gspread"><code>gspread</code></a> pour interagir avec Google Sheets.

Commencez par installer le package en utilisant l'outil <code>pip</code>.

In [0]:
!pip install --upgrade --quiet gspread

Importez la bibliothèque, authentifiez-vous et créez l'interface avec Sheets.

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Vous trouverez ci-dessous quelques exemples liés à <code>gspread</code>. D'autres exemples sont disponibles sur la page GitHub de <a href="https://github.com/burnash/gspread#more-examples"><code>gspread</code></a>.

## Créer une feuille avec des données provenant de Python

In [0]:
sh = gc.create('My cool spreadsheet')

Après avoir exécuté la cellule ci-dessus, une nouvelle feuille de calcul intitulée "My cool spreadsheet" est affichée sur le site <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Ouvrez la nouvelle feuille et ajoutez des données aléatoires.

In [0]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

## Télécharger des données d'une feuille dans Python en tant que DataFrame pandas

Relisez les données aléatoires que nous avons insérées ci-dessus, puis convertissez le résultat en <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">DataFrame pandas</a>.

In [0]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# Google Cloud Storage &#40;GCS&#41;

Pour utiliser Colaboratory avec GCS, vous devez créer un <a href="https://cloud.google.com/storage/docs/projects">projet Google Cloud</a> ou en utiliser un déjà disponible.

Spécifiez l'ID de votre projet ci-dessous :

In [0]:
project_id = 'Your_project_ID_here'

Les fichiers dans GCS sont contenus dans des <a href="https://cloud.google.com/storage/docs/key-terms#buckets">buckets</a>.

Les buckets doivent avoir un nom unique &#40;nous en générons donc un ici&#41;.

In [0]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

Pour que vous puissiez accéder à GCS, nous devons vous authentifier.

In [0]:
from google.colab import auth
auth.authenticate_user()

GCS est accessible via l'utilitaire de ligne de commande <code>gsutil</code> ou l'API Python native.

## `gsutil`

Tout d'abord, nous configurons <code>gsutil</code> pour utiliser le projet que nous avons spécifié ci-dessus à l'aide de <code>gcloud</code>.

In [0]:
!gcloud config set project {project_id}

Updated property [core/project].


Créez un fichier local à importer.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Créez un bucket dans lequel nous allons importer le fichier &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">documentation</a>&#41;.

In [0]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


Copiez le fichier dans notre nouveau bucket &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">documentation</a>&#41;.

In [0]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


Videz le contenu du fichier qui vient d'être copié afin de vous assurer que tout a été effectué correctement &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">documentation</a>&#41;.


In [0]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [0]:
#@markdown Une fois l'importation terminée, les données sont affichées dans le navigateur de stockage Cloud Console de votre projet :
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Enfin, nous téléchargeons le fichier que nous venons d'importer dans l'exemple ci-dessus. C'est aussi simple que d'inverser l'ordre dans la commande <code>gsutil cp</code>.

In [0]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Imprimez le résultat afin de vous assurer que le transfert a été effectué.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## API Python

Ces extraits sont basés sur un <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">exemple plus large</a> qui illustre d'autres utilisations de l'API.

Tout d'abord, nous créons le client de service.

In [0]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Créez un fichier local à importer.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Créez un bucket dans le projet spécifié ci-dessus.

In [0]:
# Utilisez un nom de bucket différent et unique à partir de l'exemple gsutil ci-dessus.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


Importez le fichier dans le bucket qui vient d'être créé.

In [0]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [0]:
#@markdown Une fois l'importation terminée, les données sont affichées dans le navigateur de stockage Cloud Console de votre projet :
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Téléchargez le fichier que nous venons d'importer.

In [0]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


Examinez le fichier téléchargé.


In [0]:
!cat /tmp/downloaded_from_gcs.txt

my sample file